作業: 請自行使用07_RealEstatePractice進行探索分析並做出預測結果，請在同一分jupyter notebook的最上方的block整理出:

要求項目:

1. 自行計算出尚未取log的「真實total_price」跟「預測total_price」的rmse(40%)
    整理出使用課程上或課程外學到的哪一些「前處理」、「分群」、「分類」技術(60%)

2. 加分項目:
    比較出各分類演算法在這份資料集上適用的差異與心得(20%)

3. 繳交期限: 11/26(日)

4. 資料集:以自己想練習的部分為主，想練習前處理的，可以玩未處理過的資料集(df_realestate.csv)，想多練習model的，可以玩已經做好處理的(df_realestate_processed.csv)。
    
5. 繳交方式: 將repo push到自己的github上，並提供你github中這一個notebook的連結給統一個負責人，整理完後再轉交給我。


#### AIEN0308 党榮安/ Steve Dang's Home Work Report : 
    according to main06 setting :
    X_train = X.iloc[:-1000]
    Y_train = Y.iloc[:-1000]
    Y_train = np.log(Y_train) 

    X_valid = X.iloc[-1000:]
    Y_valid = Y.iloc[-1000:]
    Y_valid = np.log(Y_valid)

    params['objective'] = 'reg:linear'
    params['eval_metric'] = 'rmse'
    params['eta'] = 0.03
    params['max_depth'] = 3

    result : without taking log [791]	train-rmse:1.25767e+07	valid-rmse:1.49863e+07



   ##### action 1 : 
        1.1 Base on above 'main06' setting, change testing data set-up to last 30% (around :-15,876 rows) as following
            X_train = X.iloc[:-15876]
            Y_train = Y.iloc[:-15876]

            X_valid = X.iloc[-15876:]
            Y_valid = Y.iloc[-15876:]

            Y_train = np.log(Y_train) 
            Y_valid = np.log(Y_valid)
            params['objective'] = 'reg:linear'
            params['eval_metric'] = 'rmse'
            params['eta'] = 0.03
            params['max_depth'] = 3
            
        result : (:-10590 valid data) [234]	train-rmse:2.12934e+07	valid-rmse:3.27672e+07
                 (:-13237 valid data) [358]	train-rmse:1.61192e+07	valid-rmse:2.95806e+07
                 (:-15876 valid data) [430]	train-rmse:1.40568e+07	valid-rmse:3.02984e+07
        conclusion : keep (:-15876 valid data)




   ##### action 2 :
        2.1 Keep 1.1 condition 

        2.2 according to 'main06' Feature Importance' chat, it only keeps 10 most importance columns as following :
            X = df_realestate_processed[['area', 'nearest_point_avg_price', 'house_year', 'income_avg', 
                  'nearest_tarin_station_distance', 'cross_RC_station_dist', 'current_division', 
                  'num_of_bus_stations_in_100m', 'cross_store_area', 'cross_through_sky_area']]
 
         result : [471]	train-rmse:1.59167e+07	valid-rmse:3.16205e+07
         conclusion : worse than action 1.  So, keep 250 column DataFrame to instead of top 10 feature importance columns



  
   ##### action 3 :
        3.1 Keep 1.1 condition

        3.2 Accordoing to 'df['area'].describe', the 'area' has wild range from 1.1 to 60000.  So, take log10 on 'area'
            to see what happen.  took X['area']=np.log(X['area']) to make 'area' around 0.09 to 6.

         result : [430]	train-rmse:1.40568e+07	valid-rmse:3.03015e+07
         conclusion : it's almost same with action 1.  So, give up log 10.



   ##### action 4 :
        4.1 Keep 1.1 condition and only change max_depth from 3 to 5 "params['max_depth'] = 5"

         result : [487]	train-rmse:9.75751e+06	valid-rmse:3.70139e+07
         conclusion : awesome.  it is improved.  keep max_depth = 5
     
     
     
   ##### action 5 :
        5.1 use pearson corrlation and found 'area' and 'Location' in top 20.
        5.2 take Top17 corelation coefficient column for Xgboost linear Reqgression.
        5.3 keep 1.1 condition plus max_depth = 5
                       Value
            1. area_square	                    0.921842
            2. area	                            0.883461
            3. cross_Xinyi_area	                0.828614
            4. cross_Xinyi_area_square	        0.824066
            5. cross_business_office_area         0.432545
            6. cross_business_office_area_square	0.394823
            7. cross_RC_area	                    0.382372
            8. cross_RC_area_square	            0.329602
            9. cross_Neihu_area	                0.328043
            10.cross_Neihu_area_square	        0.326780
            11.cross_business_area	            0.210027   
            12.cross_business_area_square         0.199408
            13.cross_resirential_area             0.199342
            14.cross_Zhongshan_area               0.191781
            15.cross_Zhongshan_area_square        0.168832
            16.cross_Songshan_area                0.168507
            17.cross_Songshan_area_square         0.165469
     
          result : [152]	train-rmse:2.3148e+07	valid-rmse:3.49974e+07
          conclusion : not good
     
     
     
     
     
   ##### action 6 :
        6.1 drop outlier 'price_per_meter'>2,500,000 by using K-MEAN.  It likes not much outliers in nearest_point_avg_price',             'house_year', 'income_avg' and 'nearest_tarin_station_distance'
        6.2 iii computer creashed when used hierarchy to make cluster. Run cluster DBSCAN is OK.
        6.3 Keep 1.1 condition and only change max_depth from 3 to 5 "params['max_depth'] = 5" 

         result : [387]	train-rmse:1.05538e+07	valid-rmse:3.72643e+07
         concliusion : train-rmse better, valid-rmse worse



   ##### action 7 :
        7.1 drop normal destribution 2 tires of following 3 columns by using 2 standard deviation as following:
            s=2
            df = df[df.area<(np.mean(df.area)+np.std(df.area)*s)]
            df = df[df.area>(np.mean(df.area)-np.std(df.area)*s)]
            df = df[df.nearest_point_avg_price<(np.mean(df.nearest_point_avg_price)+np.std(df.nearest_point_avg_price)*s)]
            df = df[df.nearest_point_avg_price>(np.mean(df.nearest_point_avg_price)-np.std(df.nearest_point_avg_price)*s)]

            df = df[df.income_avg<(np.mean(df.income_avg)+np.std(df.income_avg)*s)]
            df = df[df.income_avg>(np.mean(df.income_avg)-np.std(df.income_avg)*s)]

         result : [284]	train-rmse:7.58802e+06	valid-rmse:1.33674e+07
         conclusion : compared to action 4, it has big improved.  Awesome



   ##### action 8 :
        8.1 use Ploy to see and improvement in interactive columns.  Use following 2 columns
            poly = preprocessing.PolynomialFeatures(degree=3)
            X = df.drop(["price_per_meter", "total_price"], axis=1)
            X2 = poly.fit_transform(X[['nearest_tarin_station_distance', 'cross_business_office_area_square']])
            X = pd.DataFrame(np.concatenate([X, X2], axis=1))
            
         result : [629]	train-rmse:6.80221e+06	valid-rmse:1.3449e+07
         conclusion : improvement not much


In [2]:
import matplotlib.pyplot as plt
# from planar_utils import plot_decision_boundary, sigmoid, load_planar_dataset, load_extra_datasets
import numpy as np
import pandas as pd
import os
from collections import Counter 
import pprint
from sklearn.neighbors import KNeighborsClassifier  ## KNN
from sklearn.linear_model import LogisticRegressionCV  ## logistic regression
from sklearn.tree import DecisionTreeClassifier  ## decision tree
from sklearn.svm import SVC  ## SVM
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier  ## decision tree
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from scipy.stats import pearsonr
import math
import string
import re
from math import sqrt
import xgboost

# from preprocess import preprocess

In [3]:
if 'df_realestate.csv' not in os.listdir('data'):
    url = 'https://s3.amazonaws.com/datasets-jeremy/df_realestate.csv'  
    urllib.request.urlretrieve(url, os.path.join('data', 'df_realestate.csv')) 
    
if 'df_realestate_processed.csv' not in os.listdir('data'):
    url = 'https://s3.amazonaws.com/datasets-jeremy/df_realestate_processed.csv'  
    urllib.request.urlretrieve(url, os.path.join('data', 'df_realestate_processed.csv'))  

In [4]:
file = os.path.join('data', 'df_realestate.csv')

In [8]:
# before preprocessing
file = os.path.join('data', 'df_realestate.csv')    # make string = data/df_realestate.csv
df_realestate = pd.read_csv(file, encoding='big5')

# processed
path = "data//df_realestate_processed.csv"
df = pd.read_csv(path)

d=['area', 'nearest_point_avg_price', 'price_per_meter', 'income_avg', 'nearest_tarin_station_distance',
   'cross_RC_station_dist', 'current_division', 'num_of_bus_stations_in_100m', 'cross_store_area', 'cross_through_sky_area']
linkage = ['ward', 'average', 'complete']
epss = [0.3, 0.4, 0.5]

# for w in d:
#     datas = pd.DataFrame(df[[w, 'total_price']].values)

#     plt.figure(figsize=(16,4))

#     axes1 = plt.subplot(1, 3, 1)
#     axes1.scatter(datas[0], datas[1], s=40, cmap=plt.cm.Spectral)
#     plt.title(w+"_expected")
    
#     axes2 = plt.subplot(1, 3, 2)
#     kmeans = KMeans(n_clusters=3, random_state=0).fit_predict(datas)
#     axes2.scatter(datas[0], datas[1], c=kmeans, s=40, cmap=plt.cm.Spectral);
#     plt.title(w+'_kmeans')

# #     axes3 = plt.subplot(1, 3, 1)
# #     hierarchy = AgglomerativeClustering(linkage=linkage[0], n_clusters=3).fit_predict(datas)
# #     axes3.scatter(datas[0], datas[1], c=hierarchy, s=40, cmap=plt.cm.Spectral);

#     axes3 = plt.subplot(1, 3, 3)
#     dbscan = DBSCAN(eps=epss[2]).fit_predict(datas)
#     axes3.scatter(datas[0], datas[1], c=dbscan, s=40, cmap=plt.cm.Spectral)
#     plt.title(w+'_DBSCAN'+str(epss[0]))
    
#     plt.show()
    
# for w in datas:
#     clf = KNeighborsClassifier(n_neighbors=5)  ## 設定用最近的3個鄰居投票
#     clf.fit(datas[0], datas[1])  ## 訓練模型
#     y_pred = clf.predict(datas[0])  ## 預測模型
#     print('Accuracy',  str((Y == y_pred).sum()/ datas[0].shape[0]*100)+"%")  ## 計算精準度
    
#     plot_decision_boundary(lambda x: clf.predict(x), datas[0].T, datas[1])  ## 視覺化分類器的分類結果
#     plt.title(name+'_knn(' + str((Y == y_pred).sum()/ dadas[0].shape[0]*100)+"%)")
#     plt.show()

# df_realestate_processed = df_realestate_processed[df_realestate_processed.area<50000]
# df_realestate_processed = df_realestate_processed[df_realestate_processed.nearest_point_avg_price<2500000]
s=2
df = df[df.area<(np.mean(df.area)+np.std(df.area)*s)]
df = df[df.area>(np.mean(df.area)-np.std(df.area)*s)]

df = df[df.nearest_point_avg_price<(np.mean(df.nearest_point_avg_price)+np.std(df.nearest_point_avg_price)*s)]
df = df[df.nearest_point_avg_price>(np.mean(df.nearest_point_avg_price)-np.std(df.nearest_point_avg_price)*s)]

df = df[df.income_avg<(np.mean(df.income_avg)+np.std(df.income_avg)*s)]
df = df[df.income_avg>(np.mean(df.income_avg)-np.std(df.income_avg)*s)]


poly = preprocessing.PolynomialFeatures(degree=3)
X = df.drop(["price_per_meter", "total_price"], axis=1)
# X2 = poly.fit_transform(X[['nearest_tarin_station_distance', 'cross_business_office_area_square']])

# X = pd.DataFrame(np.concatenate([X, X2], axis=1))
# print(X.shape)

# X = df_realestate_processed[['area', 'nearest_point_avg_price', 'house_year', 'income_avg', 'nearest_tarin_station_distance',
#                             'cross_RC_station_dist', 'current_division', 'num_of_bus_stations_in_100m', 'cross_store_area',
#                             'cross_through_sky_area']]
# X = df_realestate_processed[['area', 'area_square', 'cross_Xinyi_area', 'cross_Xinyi_area_square', 'cross_business_office_area', 
#                            'cross_business_office_area_square', 'cross_RC_area_square', 'cross_RC_area', 'cross_Neihu_area_square', 
#                           'cross_Neihu_area', 'cross_business_area_square', 'cross_business_area', 'cross_resirential_area', 
#                             'cross_Zhongshan_area_square', 'cross_Zhongshan_area']]

# X['area']=np.log(X['area'])

# print(X.head())

# print(df_realestate_processed.head())

Y = df['total_price']
# Y = df_realestate_processed['total_price']
# features = X.columns.tolist()

# print(features)
# print(target)
# correlations = {}

# Pearson correlation coefficient
# for f in features:
#     key=f+'vs. price'
#     correlations[f] = pearsonr(X[f],Y)[0]
# data_correlations = pd.DataFrame(correlations, index=['Value']).T
# x = data_correlations['Value'].sort_values(ascending=False)
# print(x)


In [10]:
# Start Your Code Here  DataShape (52950, 254)
# it is better tp enlarge 1000 to 10000 for vlidation data size
# X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=0.3, random_state=42)
# X_train.head()

X_train = X.iloc[:-15876]
Y_train = Y.iloc[:-15876]
# Y_train = np.log(Y_train) 

X_valid = X.iloc[-15876:]
Y_valid = Y.iloc[-15876:]
# Y_valid = np.log(Y_valid) 

params = {}

params['objective'] = 'reg:linear'
# params['objective'] = 'reg:gamma'

params['eval_metric'] = 'rmse'

params['eta'] = 0.03
params['max_depth'] = 5
d_train = xgboost.DMatrix(X_train, label=Y_train)
d_valid = xgboost.DMatrix(X_valid, label=Y_valid)
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
bst = xgboost.train(params, d_train, 3000, watchlist, early_stopping_rounds=10, verbose_eval=10)
Y_pred = bst.predict(xgboost.DMatrix(X_valid))


[10:50:17] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[0]	train-rmse:4.27848e+07	valid-rmse:3.97114e+07
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[10:50:17] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[10:50:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[10:50:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[10:50:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[10:50:18] C:\Users\Administrator\Desktop\xgboost\

[10:50:22] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[10:50:22] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[10:50:22] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[10:50:22] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[10:50:22] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[10:50:22] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[10:50:22] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[10:50:26] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[10:50:26] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[10:50:26] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[10:50:26] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[10:50:27] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[10:50:27] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[110]	train-rmse:9.49128e+06	valid-rmse:1.44854e+07
[10:50:27] C:\Users\Administrator\Desktop\

[10:50:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[10:50:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[10:50:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[160]	train-rmse:8.61045e+06	valid-rmse:1.40201e+07
[10:50:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[10:50:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 0 pruned nodes, max_depth=5
[10:50:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[10:50:31] C:\Users\Administrator\Desktop\

[10:50:35] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[10:50:35] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=5
[10:50:35] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 0 pruned nodes, max_depth=5
[10:50:35] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[10:50:35] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=5
[10:50:35] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=5
[10:50:35] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

...

[10:50:48] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=5
[370]	train-rmse:7.54695e+06	valid-rmse:1.36333e+07
[10:50:48] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[10:50:48] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[10:50:48] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=5
[10:50:48] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=5
[10:50:48] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[10:50:49] C:\Users\Administrator\Desktop\

[10:50:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[10:50:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 0 pruned nodes, max_depth=5
[10:50:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 0 pruned nodes, max_depth=5
[10:50:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[10:50:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[10:50:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=5
[10:50:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[10:50:57] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[10:50:57] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=5
[10:50:57] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=5
[10:50:57] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=5
[10:50:57] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[480]	train-rmse:7.21584e+06	valid-rmse:1.3533e+07
[10:50:57] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=5
[10:50:57] C:\Users\Administrator\Desktop\x

[10:51:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=5
[10:51:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=5
[10:51:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[10:51:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=5
[10:51:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=5
[10:51:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[10:51:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[10:51:10] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=5
[10:51:10] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=5
[10:51:10] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[10:51:11] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=5
[10:51:11] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[10:51:11] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
Stopping. Best iteration:
[629]	train-rmse:6.80221e+06	valid-rmse:1.3449e+07



In [ ]:
xgboost.plot_importance(bst, max_num_features=15)
#!==============your works ends================!#

plt.show()

In [2]:
plt.scatter(X.area, Y, c = "blue", marker = "s")
# plt.scatter(X.nearest_point_avg_price, Y, c = "blue", marker = "s")
# plt.scatter(X.house_year, Y, c = "blue", marker = "s")
# plt.scatter(X.income_avg, Y, c = "blue", marker = "s")
# plt.scatter(X.nearest_tarin_station_distance, Y, c = "blue", marker = "s")

plt.title("GrLivArea vs SalePrice")
plt.xlabel("GrLivArea")
plt.ylabel("SalePrice")
plt.show()

df_result = pd.DataFrame()

# 1. 使用X_valid去評價此模型
# 2. 使用 ['predict', 'truth', 'error'] 三個欄位的DataFrame去使決畫呈現預測結果
#    (1). 請注意與測結果(Y_pred)與真實值(Y_valid)都必須取exp方能反映實際情況
#    (2). error 請使用計算np.abs(predict-truth)/truth計算誤差百分比
#=============your works starts===============#
Y_pred = bst.predict(xgboost.DMatrix(X_valid))
df_result['predict'] = np.exp(Y_pred)
df_result['truth'] = np.exp((list(Y_valid)))
df_result['error'] = df_result.apply(lambda x:np.abs(x['predict'] - x['truth']) , axis=1)
df_result['error%'] = df_result.apply(lambda x:np.abs(x['predict'] - x['truth']) / x['truth']*100, axis=1)
df_result_sort = df_result.sort_values('truth')
#==============your works ends================#

print(df_result.head(10))
print('mean of abs(error) :', df_result['error'].mean())
rms = sqrt(mean_squared_error(df_result['truth'], df_result['predict']))
print('rmse :', rms)

NameError: name 'plt' is not defined